# 人口データの取得・可視化・分析


In [41]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得に用いるライブラリ
import requests
# データ処理に用いるライブラリ
import numpy as np
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [40]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
)

# estat.py から get_metainfo などの関数を直接インポートしているので、後のセルで関数名だけで呼び出して使うことができるようになる

In [44]:
# .envファイルから環境変数を読む
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

In [45]:
current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch07").resolve()

## 人口と国勢調査

- [国勢調査](https://www.e-stat.go.jp/statistics/00200521)
- [令和2 年国勢調査（2020 年）の公表](https://www.stat.go.jp/data/kokusei/2020/pdf/schedule.pdf)


In [5]:
pop_statsDataId = "0003410380"
pop_meta = get_metainfo(appId, pop_statsDataId)                     # estat.py で定義した get_metainfo 関数を呼び出して使う
pop_metadata = pop_meta["GET_META_INFO"]["METADATA_INF"]
pop_total_num = pop_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
pop_total_num

3752

In [6]:
pop_data = get_statsdata(appId, pop_statsDataId)                    # estat.py で定義した get_statsdata 関数を呼び出して使う
pop_value = colname_to_japanese(cleansing_statsdata(pop_data))      # estat.py で定義した cleansing_statsdata 関数を呼び出して使う
pop_value.columns

Index(['表章項目コード', '男女_時系列コード', '年齢（５歳階級）_時系列コード', '時間軸（調査年）コード', '単位', '値',
       '表章項目', '表章項目階層レベル', '表章項目単位', '男女_時系列', '男女_時系列階層レベル', '年齢（５歳階級）_時系列',
       '年齢（５歳階級）_時系列階層レベル', '時間軸（調査年）', '時間軸（調査年）階層レベル'],
      dtype='object')

In [7]:
pop_value

,表章項目コード,男女_時系列コード,年齢（５歳階級）_時系列コード,時間軸（調査年）コード,単位,値,表章項目,表章項目階層レベル,表章項目単位,男女_時系列,男女_時系列階層レベル,年齢（５歳階級）_時系列,年齢（５歳階級）_時系列階層レベル,時間軸（調査年）,時間軸（調査年）階層レベル
0,020,100,100,1920000000,人,5.596305e+07,人口,,人,総数,1,総数,1,1920年,1
1,020,100,100,1925000000,人,5.973682e+07,人口,,人,総数,1,総数,1,1925年,1
2,020,100,100,1930000000,人,6.445000e+07,人口,,人,総数,1,総数,1,1930年,1
3,020,100,100,1935000000,人,6.925415e+07,人口,,人,総数,1,総数,1,1935年,1
4,020,100,100,1940000000,人,7.307507e+07,人口,,人,総数,1,総数,1,1940年,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3747,1120,100,400,2010000000,NaN,7.433806e+01,人口性比,,NaN,総数,1,（再掲）65歳以上,1,2010年,1
3748,1120,100,400,2015000000,NaN,7.631976e+01,人口性比,,NaN,総数,1,（再掲）65歳以上,1,2015年,1
3749,1120,100,400,2020000000,NaN,7.675986e+01,人口性比,,NaN,総数,1,（再掲）65歳以上,1,2020年,1
3750,1120,100,400,2015000010,NaN,7.643550e+01,人口性比,,NaN,総数,1,（再掲）65歳以上,1,2015年_不詳補完値,1


### 人口ピラミッド


In [8]:
pop_value["男女_時系列"].unique()

array(['総数', '男', '女'], dtype=object)

In [9]:
# 1. 最終的に取り出したい「列」の名前をリストにまとめる
# （年齢コード、年齢名称、性別、人口の数値）
pyramid_cols = ["年齢（５歳階級）_時系列コード", "年齢（５歳階級）_時系列", "男女_時系列", "値"]

# 2. 抽出するための「条件」を作っていく（pyramid_cond）
# まず、項目が「人口」であるものだけを指定
pyramid_cond = pop_value["表章項目"] == "人口"

# さらに条件を追加：性別の「総数」は除外（男・女に分けたいので）
pyramid_cond &= pop_value["男女_時系列"] != "総数"

# さらに条件を追加：年齢の「総数」も除外
pyramid_cond &= pop_value["年齢（５歳階級）_時系列"] != "総数"

# さらに条件を追加：名前に「再掲」が含まれる行（重複データ）を除外
pyramid_cond &= ~ pop_value["年齢（５歳階級）_時系列"].str.contains("再掲")

# さらに条件を追加：2020年の「不詳補完値（詳細な集計データ）」のみを対象にする
pyramid_cond &= pop_value["時間軸（調査年）"] == "2020年_不詳補完値"

# 3. 作成した全ての条件（pyramid_cond）に一致する行と、
# 選んだ列（pyramid_cols）だけを抜き出して、新しい変数 pyramid_df に入れる
pyramid_df = pop_value.loc[pyramid_cond, pyramid_cols]

In [10]:
pop_pyramid = pyramid_df.sort_values("年齢（５歳階級）_時系列コード").assign(**
    {"人口": pyramid_df["値"].mask(pyramid_df["男女_時系列"] == "男", -pyramid_df["値"])}
).rename(columns={"年齢（５歳階級）_時系列": "年齢（５歳階級）", "男女_時系列": "性別"})
pop_pyramid.head()

,年齢（５歳階級）_時系列コード,年齢（５歳階級）,性別,値,人口
581,110,０～４歳,男,2324576.0,-2324576.0
1117,110,０～４歳,女,2216784.0,2216784.0
604,120,５～９歳,男,2619882.0,-2619882.0
1140,120,５～９歳,女,2494293.0,2494293.0
627,130,10～14歳,男,2755578.0,-2755578.0


In [11]:
pyramid_fig = px.bar(
    pop_pyramid, 
    x="人口", 
    y="年齢（５歳階級）", 
    color="性別", 
    orientation="h"  # "h"は水平方向(横棒グラフ)
)
pyramid_fig.update_layout(width=900, height=600, font={"size": 14},
    xaxis=go.layout.XAxis(
       tickvals=[-4_000_000, -2_000_000, 0, 2_000_000, 4_000_000],
       ticktext=["400万","200万", "0", "200万", "400万"],
    ))
pyramid_fig.update_layout(         # タイトルを付ける
    title_text="人口ピラミッド",    # タイトルの表示名 
    title_x=0.5                    # 真ん中に表示
)
pyramid_fig.show()

### 人口推移

In [12]:
pop_cols = ["時間軸（調査年）コード", "年齢（５歳階級）_時系列コード", "年齢（５歳階級）_時系列", "値"]
pop_cond = pop_value["表章項目"] == "人口"
pop_cond &= pop_value["男女_時系列"] == "総数"
pop_cond &= pop_value["年齢（５歳階級）_時系列"] != "総数"
pop_cond &= ~ pop_value["年齢（５歳階級）_時系列"].str.contains("再掲")
pop_cond &= ~ pop_value["時間軸（調査年）"].str.contains("不詳補完値")
pop_df = pop_value.loc[pop_cond, pop_cols]

In [13]:
pop_df = pop_df.assign(**{
    "年": pop_df["時間軸（調査年）コード"].astype(int) // 1_000_000,
    "年齢（５歳階級）": pop_df["年齢（５歳階級）_時系列コード"] + "_" + pop_df["年齢（５歳階級）_時系列"]
})

In [14]:
# 1. 3つの異なるカラーテーマから、必要な数だけ色を抜き出して合体させています
# 「dense（濃い青系）」から最初の3色を取得 -> 最初の0～14歳に適用される
pop_colors = px.colors.sequential.dense[:3]

# 「matter（赤・桃系）」から最初の10色を追加 -> 15～64歳に適用される
pop_colors += px.colors.sequential.matter[:10]

# 「deep（深い緑・青系）」から最初の11色を追加 -> 65歳以上に適用される
pop_colors += px.colors.sequential.deep[:11]
# これで、合計24色（3 + 10 + 11）のカラフルな特製リストができました。

# 2. 作成した色のリスト（pop_colors）を使ってグラフを描画します
pop_fig = px.bar(
    pop_df, 
    x="年", 
    y="値", 
    color="年齢（５歳階級）",           # 「年齢」の項目ごとに色を変える
    color_discrete_sequence=pop_colors # さっき作った24色のリストを順番に使う
)

# 3. グラフの見た目を整える設定
pop_fig.update_layout(
    width=900, height=700, font={"size": 14},
    xaxis={"ticksuffix": "年", "title": None}, # X軸に「年」をつける
    yaxis={"tickformat": ",.0f", "ticksuffix": "人", "title": None} # Y軸にカンマと「人」をつける
)

# 4. タイトルの設定（中央寄せ）
pop_fig.update_layout(
    title_text="人口推移",
    title_x=0.5 # 0.5は中央
)

pop_fig.show()

## 人口動態

- [人口動態調査](https://www.mhlw.go.jp/toukei/list/81-1.html)
- [公表](https://www.mhlw.go.jp/toukei/kouhyou/e-stat_81-1.xml)


In [15]:
vital_statsDataId = "0003411561"
vital_meta = get_metainfo(appId, vital_statsDataId)     # get_metainfo 関数の利用
vital_metadata = vital_meta["GET_META_INFO"]["METADATA_INF"]
vital_total_num = vital_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
vital_total_num

3294

In [16]:
vital_data = get_statsdata(appId, vital_statsDataId)    # get_statsdata 関数の利用
vital_value_df = colname_to_japanese(cleansing_statsdata(vital_data))
vital_value_df.columns

Index(['人口動態総覧コード', '時間軸(年次)コード', '単位', '値', '人口動態総覧', '人口動態総覧階層レベル',
       '人口動態総覧単位', '人口動態総覧親コード', '時間軸(年次)', '時間軸(年次)階層レベル'],
      dtype='object')

In [17]:
vital_value_df

,人口動態総覧コード,時間軸(年次)コード,単位,値,人口動態総覧,人口動態総覧階層レベル,人口動態総覧単位,人口動態総覧親コード,時間軸(年次),時間軸(年次)階層レベル
0,00110,2023000000,人,727288.00,出生数,1,人,NaN,2023年,1
1,00110,2022000000,人,770759.00,出生数,1,人,NaN,2022年,1
2,00110,2021000000,人,811622.00,出生数,1,人,NaN,2021年,1
3,00110,2020000000,人,840835.00,出生数,1,人,NaN,2020年,1
4,00110,2019000000,人,865239.00,出生数,1,人,NaN,2019年,1
...,...,...,...,...,...,...,...,...,...,...
3289,00430,1903000000,人口千対,1.44,離婚率,1,人口千対,NaN,1903年,1
3290,00430,1902000000,人口千対,1.43,離婚率,1,人口千対,NaN,1902年,1
3291,00430,1901000000,人口千対,1.43,離婚率,1,人口千対,NaN,1901年,1
3292,00430,1900000000,人口千対,1.46,離婚率,1,人口千対,NaN,1900年,1


### 出生数と死亡数


In [18]:
# 1. 「年」という新しい列を追加する
# e-Statの「時間軸コード」は、例えば2020年なら "2020000000" という10桁の数値/文字列になっています。
# これを「2020」という数値にするために、以下の処理をしています：
# .astype(int) で整数に変換し、
# // 1000000 で「100万」で割って余りを切り捨てる（＝最初の4〜5桁だけを取り出す）
vital_value = vital_value_df.assign(**{
    "年": vital_value_df["時間軸(年次)コード"].astype(int) // 1_000_000
})

# 2. 欲しいデータだけに絞り込むための「条件」を作る
# 「人口動態総覧」という列の中に「出生数」か「死亡数」が入っている行だけを探します
# vital_cond にはTrueとFalseが入る
vital_cond = vital_value["人口動態総覧"].isin(["出生数", "死亡数"])

# 3. 条件に合う「行」と、必要な3つの「列」だけを抜き出す
# .loc[条件, [列名のリスト]] の形になっています -> vital_condがTrueの行について、["年", "人口動態総覧", "値"]の列を抽出して vital_dfに代入している
vital_df = vital_value.loc[vital_cond, ["年", "人口動態総覧", "値"]]

# 4. 出来上がったデータの先頭5行を表示
vital_df.head()

,年,人口動態総覧,値
0,2023,出生数,727288.0
1,2022,出生数,770759.0
2,2021,出生数,811622.0
3,2020,出生数,840835.0
4,2019,出生数,865239.0


In [19]:
print(vital_cond)

0        True
1        True
2        True
3        True
4        True
        ...  
3289    False
3290    False
3291    False
3292    False
3293    False
Name: 人口動態総覧, Length: 3294, dtype: bool


In [20]:
vital_value["人口動態総覧"].unique()

array(['出生数', '死亡数', '乳児死亡数', '新生児死亡数', '自然増減数', '死産数', '自然死産数', '人工死産数',
       '周産期死亡数', '満22週以後の死産数', '早期新生児死亡', '婚姻件数', '離婚件数', '出生率',
       '合計特殊出生率', '死亡率', '乳児死亡率', '新生児死亡率', '自然増減率', '死産率', '自然死産率',
       '人工死産率', '周産期死亡率', '満22週以後の死産率', '早期新生児死亡率', '婚姻率', '離婚率'],
      dtype=object)

In [21]:
vital_delta = vital_value.loc[vital_value["人口動態総覧"] == "自然増減数", ["年", "値"]]
delta_pop = vital_delta.sort_values("年").rename(columns={"値": "自然増減数"})

In [22]:
vital_fig = px.line(vital_df, x="年", y="値", color="人口動態総覧", markers=True)   # vital_dfには、人口動態総覧に出生数と死亡数が格納されている年、人口動態総覧（出生数・死亡数の別）、値が格納されている
vital_fig.add_bar(x=delta_pop["年"], y=delta_pop["自然増減数"], name="自然増減数")
vital_fig.update_layout(width=1200, height=650, font={"size": 20},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "人", "title": None}
)
vital_fig.update_layout(
    title_text="出生数、死亡数、および自然増減数",
    title_x=0.5
)
vital_fig.show()

### 合計特殊出生率

-[人口置換水準](https://www.ipss.go.jp/syoushika/tohkei/Popular/Popular2020.asp?chap=0)


In [23]:
vital_value

,人口動態総覧コード,時間軸(年次)コード,単位,値,人口動態総覧,人口動態総覧階層レベル,人口動態総覧単位,人口動態総覧親コード,時間軸(年次),時間軸(年次)階層レベル,年
0,00110,2023000000,人,727288.00,出生数,1,人,NaN,2023年,1,2023
1,00110,2022000000,人,770759.00,出生数,1,人,NaN,2022年,1,2022
2,00110,2021000000,人,811622.00,出生数,1,人,NaN,2021年,1,2021
3,00110,2020000000,人,840835.00,出生数,1,人,NaN,2020年,1,2020
4,00110,2019000000,人,865239.00,出生数,1,人,NaN,2019年,1,2019
...,...,...,...,...,...,...,...,...,...,...,...
3289,00430,1903000000,人口千対,1.44,離婚率,1,人口千対,NaN,1903年,1,1903
3290,00430,1902000000,人口千対,1.43,離婚率,1,人口千対,NaN,1902年,1,1902
3291,00430,1901000000,人口千対,1.43,離婚率,1,人口千対,NaN,1901年,1,1901
3292,00430,1900000000,人口千対,1.46,離婚率,1,人口千対,NaN,1900年,1,1900


In [24]:
tfr_cond = vital_value["人口動態総覧"] == "合計特殊出生率"

# tfr_cond = tfr_cond & vital_value["値"].notna()を短く書いたものが下のコード
# 人口動態総覧における合計特殊出生率の行で値がNanでないものをtfr_condに代入し直す
tfr_cond &= vital_value["値"].notna()

tfr_df = vital_value.loc[tfr_cond, ["年", "人口動態総覧", "値"]]
tfr_df.head()

,年,人口動態総覧,値
1708,2023,合計特殊出生率,1.20
1709,2022,合計特殊出生率,1.26
1710,2021,合計特殊出生率,1.30
1711,2020,合計特殊出生率,1.33
1712,2019,合計特殊出生率,1.36


In [25]:
tfr_fig = px.line(tfr_df, x="年", y="値", color="人口動態総覧")
tfr_fig.add_trace(go.Scatter(
    # データの「年」の最小と最大を自動で使う
x = [tfr_df["年"].min(), tfr_df["年"].max()], 
    y=[2.07, 2.07],
    mode="lines", 
    line={"dash": "dash"},
    name="人口置換水準"))
tfr_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis= {"ticksuffix": "人", "title": None}
)
tfr_fig.show()

In [26]:
tfr_fig = px.line(tfr_df, x="年", y="値", color="人口動態総覧")
tfr_fig.add_trace(go.Scatter(
    x = [tfr_df["年"].min(), tfr_df["年"].max()],  # データの「年」の最小と最大を自動で使う
    y=[2.07, 2.07],
    mode="lines", 
    line={"dash": "dash"},
    name="人口置換水準"))
tfr_fig.update_layout(width=900, height=450, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis= {"ticksuffix": "人", "title": None}
)
tfr_fig.show()

### 婚姻件数と離婚件数


In [27]:
divorce_statsDataId = "0003411864"
divorce_meta = get_metainfo(appId, divorce_statsDataId)
divorce_metadata = divorce_meta["GET_META_INFO"]["METADATA_INF"]
divorce_total_num = divorce_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
divorce_total_num

1085

In [28]:
divorce_data = get_statsdata(appId, divorce_statsDataId)
divorce_value = colname_to_japanese(cleansing_statsdata(divorce_data))
divorce_value.columns

Index(['同居期間コード', '時間軸(年次)コード', '単位', '値', '同居期間', '同居期間階層レベル', '同居期間単位',
       '同居期間親コード', '時間軸(年次)', '時間軸(年次)階層レベル'],
      dtype='object')

In [29]:
divorce_value["単位"].unique()

array(['件', '%', '年'], dtype=object)

In [30]:
divorce_value["同居期間"].unique()

array(['離婚件数_総数', '離婚件数_5年未満', '離婚件数_1年未満', '離婚件数_1～2年未満', '離婚件数_2～3年未満',
       '離婚件数_3～4年未満', '離婚件数_4～5年未満', '離婚件数_5～10年未満', '離婚件数_10～15年未満',
       '離婚件数_15～20年未満', '離婚件数_20～25年未満', '離婚件数_25～30年未満', '離婚件数_30～35年未満',
       '離婚件数_35年以上', '離婚件数_不詳', '百分率_総数', '百分率_5年未満', '百分率_1年未満',
       '百分率_1～2年未満', '百分率_2～3年未満', '百分率_3～4年未満', '百分率_4～5年未満',
       '百分率_5～10年未満', '百分率_10～15年未満', '百分率_15～20年未満', '百分率_20～25年未満',
       '百分率_25～30年未満', '百分率_30～35年未満', '百分率_35年以上', '百分率_不詳', '平均同居期間'],
      dtype=object)

In [31]:
divorce_cols = ["時間軸(年次)コード", "同居期間コード", "同居期間", "値"]
divorce_cond = divorce_value["単位"] == "件"
divorce_cond &= ~divorce_value["同居期間"].isin(["離婚件数_総数", "離婚件数_5年未満"])
divorce_df = divorce_value.loc[divorce_cond, divorce_cols]

In [32]:
# このセルは実行回数分だけ同居期間コードの後ろ3桁が追加されてしまうので注意

divorce_df = divorce_df.assign(**{
    "同居期間": divorce_df["同居期間コード"].str[-3:] + "_" + divorce_df["同居期間"],
    "年": divorce_df["時間軸(年次)コード"].astype(int) // 1_000_000
})

In [33]:
pd.DataFrame(divorce_df)

,時間軸(年次)コード,同居期間コード,同居期間,値,年
70,2023000000,00120,120_離婚件数_1年未満,8814.0,2023
71,2022000000,00120,120_離婚件数_1年未満,8971.0,2022
72,2021000000,00120,120_離婚件数_1年未満,9853.0,2021
73,2020000000,00120,120_離婚件数_1年未満,10973.0,2020
74,2019000000,00120,120_離婚件数_1年未満,11834.0,2019
...,...,...,...,...,...
520,1965000000,00240,240_離婚件数_不詳,75.0,1965
521,1960000000,00240,240_離婚件数_不詳,51.0,1960
522,1955000000,00240,240_離婚件数_不詳,53.0,1955
523,1950000000,00240,240_離婚件数_不詳,939.0,1950


In [34]:
marriage_cond = vital_value["人口動態総覧"].isin(["婚姻件数", "離婚件数"])
marriage_cond &= vital_value["年"] >= 1947
marriage_df = vital_value.loc[marriage_cond, ["年", "人口動態総覧", "値"]]
pivot_df = marriage_df.pivot(columns="人口動態総覧", index="年", values="値")
pivot_df.head()

人口動態総覧,婚姻件数,離婚件数
年,,
1947,934170.0,79551.0
1948,953999.0,79032.0
1949,842170.0,82575.0
1950,715081.0,83689.0
1951,671905.0,82331.0


In [35]:
# サブプロットの作成
marriage_fig = make_subplots(rows=1, cols=2, subplot_titles=("婚姻・離婚件数", "離婚内訳"))
# 左側の婚姻・離婚件数グラフ
marriage_fig.add_trace(go.Scatter(
    x=pivot_df.index, 
    y=pivot_df["婚姻件数"],
    mode="lines", 
    name="婚姻件数"
), row=1, col=1)
marriage_fig.add_trace(go.Scatter(
    x=pivot_df.index, 
    y=pivot_df["離婚件数"],
    mode="lines", 
    name="離婚件数"
), row=1, col=1)    #1行1列目に描画
# 右側の離婚内訳エリアグラフ(matterのカラーが不足するので一つカラーコード追加)
divorce_colors = px.colors.sequential.matter + ["#EF553B"]
for n, t in enumerate(divorce_df["同居期間"].unique()):
    df_filtered = divorce_df[divorce_df["同居期間"] == t]
    marriage_fig.add_trace(go.Scatter(
        x=df_filtered["年"], 
        y=df_filtered["値"],
        name=t, 
        marker={"color": divorce_colors[n]},
        stackgroup="divorcegroup"
    ), row=1, col=2)    #1行2列目に描画
marriage_fig.update_layout(width=1200, height=550, font={"size": 14})
# 左側グラフのX軸とY軸の設定（row=1, col=1)
marriage_fig.update_xaxes(ticksuffix="年", row=1, col=1)
marriage_fig.update_yaxes(tickformat=",", ticksuffix="人", row=1, col=1)
# 右側グラフのX軸とY軸の設定(row=1, col=2)
marriage_fig.update_xaxes(ticksuffix="年", row=1, col=2)
marriage_fig.update_yaxes(tickformat=",", ticksuffix="人", row=1, col=2)
marriage_fig.show()

## 人口推計

- [人口推計](https://www.stat.go.jp/data/jinsui/index.html)
- [公表](https://www.stat.go.jp/data/kouhyou/e-stat_jinsui.xml)


In [37]:
pop_estimates_statsDataId = "0003448228"
estimates_meta = get_metainfo(appId, pop_estimates_statsDataId)
estimates_metadata = estimates_meta["GET_META_INFO"]["METADATA_INF"]
estimates_total_num = estimates_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
estimates_total_num

3876

In [ ]:
estimates_data = get_statsdata(appId, pop_estimates_statsDataId)    # appIdはセル[3]において環境変数としてAPIキーを格納している変数
estimates_value = colname_to_japanese(cleansing_statsdata(estimates_data))
estimates_value.columns

Index(['男女別・性比コード', '人口コード', '年齢各歳コード', '全国コード', '時間軸（年月日現在）コード', '単位', '値',
       '男女別・性比', '男女別・性比階層レベル', '男女別・性比単位', '人口', '人口階層レベル', '年齢各歳',
       '年齢各歳階層レベル', '全国', '全国階層レベル', '時間軸（年月日現在）', '時間軸（年月日現在）階層レベル'],
      dtype='object')

In [48]:
estimates_value["男女別・性比"].unique()

array(['男女計', '男', '女', '人口性比'], dtype=object)

In [44]:
estimates_value["人口"].unique()

array(['総人口', '日本人人口'], dtype=object)

In [45]:
estimates_value["時間軸（年月日現在）"].unique()

array(['2020年10月1日現在', '2021年10月1日現在', '2022年10月1日現在', '2023年10月1日現在',
       '2024年10月1日現在'], dtype=object)

In [46]:
estimates_value["年齢各歳"].unique()

array(['総数', '0歳', '1歳', '2歳', '3歳', '4歳', '5歳', '6歳', '7歳', '8歳', '9歳',
       '10歳', '11歳', '12歳', '13歳', '14歳', '15歳', '16歳', '17歳', '18歳',
       '19歳', '20歳', '21歳', '22歳', '23歳', '24歳', '25歳', '26歳', '27歳',
       '28歳', '29歳', '30歳', '31歳', '32歳', '33歳', '34歳', '35歳', '36歳',
       '37歳', '38歳', '39歳', '40歳', '41歳', '42歳', '43歳', '44歳', '45歳',
       '46歳', '47歳', '48歳', '49歳', '50歳', '51歳', '52歳', '53歳', '54歳',
       '55歳', '56歳', '57歳', '58歳', '59歳', '60歳', '61歳', '62歳', '63歳',
       '64歳', '65歳', '66歳', '67歳', '68歳', '69歳', '70歳', '71歳', '72歳',
       '73歳', '74歳', '75歳', '76歳', '77歳', '78歳', '79歳', '80歳', '81歳',
       '82歳', '83歳', '84歳', '85歳', '86歳', '87歳', '88歳', '89歳', '90歳',
       '91歳', '92歳', '93歳', '94歳', '95歳', '96歳', '97歳', '98歳', '99歳',
       '100歳以上'], dtype=object)

In [49]:
# 1. 抽出したい列（デカラム名）をリストで定義
estimates_cols = ["男女別・性比", "年齢各歳コード", "年齢各歳", "値"]

# 2. 抽出条件（フィルター）の作成
# 男女別・性比が「男」または「女」であること（合計などを除外）
estimates_cond = estimates_value["男女別・性比"].isin(["男", "女"])

# 人口の種類が「総人口」であること
estimates_cond &= estimates_value["人口"] == "総人口"

# 調査時点が「2024年10月1日現在」であること
estimates_cond &= estimates_value["時間軸（年月日現在）"] == "2024年10月1日現在"

# 年齢が「総数」（全年齢合計）ではないこと（各歳ごとのデータが欲しいため）
estimates_cond &= estimates_value["年齢各歳"] != "総数"

# 3. 定義した条件と列を指定して、新しいデータフレーム（estimates_df）を作成
estimates_df = estimates_value.loc[estimates_cond, estimates_cols]

In [40]:
estimates_df

,男女別・性比,年齢各歳コード,年齢各歳,値
1029,男,01001,0歳,367.0
1034,男,01002,1歳,388.0
1039,男,01003,2歳,409.0
1044,男,01004,3歳,424.0
1049,男,01005,4歳,427.0
...,...,...,...,...
2529,女,01097,96歳,135.0
2534,女,01098,97歳,90.0
2539,女,01099,98歳,66.0
2544,女,01100,99歳,44.0


In [51]:
# assignメソッドを使って、新しい列「人口」と「年齢」を同時に追加（または更新）します
estimates_df = estimates_df.assign(**{
    # 1. 「人口」列の作成：
    # 男女別が「男」の場合は、値をマイナス（負数）に変換します。
    # これにより、グラフ化した時に左側に伸びるようになります。（女性はプラスのまま右側へ）
    "人口": estimates_df["値"].mask(estimates_df["男女別・性比"] == "男", -estimates_df["値"]),

    # 2. 「年齢」列の作成：
    # 「年齢各歳コード」という文字列の末尾3桁を切り出し（str[-3:]）、
    # 数値に変換（astype(int)）してから 1 を引きます。
    # 例：コード "001" → 数値 1 → 年齢 0歳 となるように調整しています。
    "年齢": estimates_df["年齢各歳コード"].str[-3:].astype(int) - 1
})

In [53]:
estimates_fig = px.bar(estimates_df, x="人口", y="年齢", color="男女別・性比", orientation="h")
estimates_fig.update_layout(width=1000, height=450, font={"size": 18},
    xaxis=go.layout.XAxis(
       tickvals=[-1000, -500, 0, 500, 1000],
       ticktext=["1000千","500千", "0", "500千", "1000千"],
    ),
    yaxis={"ticksuffix": "歳", "title": None}
)
estimates_fig.update_layout(
    title_text="人口ピラミッド",
    title_x=0.5
)
estimates_fig.show()

## 将来推計人口

- [国立社会保障・人口問題研究](https://www.ipss.go.jp/)
- [国立社会保障・人口問題研究所のサイトの「将来推計人口・世帯数」のページ](https://www.ipss.go.jp/syoushika/tohkei/Mainmenu.asp)
- [著作権・リンク許可について](https://www.ipss.go.jp/site-ad/link/anlink.html)
- [openpyxl](https://openpyxl.readthedocs.io/en/stable/)


In [ ]:
#!pip install openpyxl

In [56]:
pop_projections_df = pd.read_excel(data_path / "1-1.xlsx", header=[0, 1], skiprows=2, skipfooter=2)

In [63]:
url_path = "https://www.ipss.go.jp/pp-zenkoku/j/zenkoku2023/db_zenkoku2023/s_tables/1-1.xlsx"
pop_projections_df = pd.read_excel(url_path, header=[0, 1], skiprows=2, skipfooter=2)
pop_projections_df.head()

年　次                    人　口（1,000人）                        \
  Unnamed: 0_level_1 Unnamed: 1_level_1         総　数      0～14歳     15～64歳   
0               令和 2               2020  126146.099  15031.602  75087.865   
1                  3               2021  125526.631  14792.434  74508.070   
2                  4               2022  124977.644  14515.325  74196.042   
3                  5               2023  124407.998  14202.024  73858.119   
4                  6               2024  123843.669  13914.831  73465.788   

             割　合（％）               
       65歳以上  0～14歳 15～64歳 65歳以上  
0  36026.632   11.9   59.5  28.6  
1  36226.126   11.8   59.4  28.9  
2  36266.277   11.6   59.4  29.0  
3  36347.856   11.4   59.4  29.2  
4  36463.050   11.2   59.3  29.4

In [64]:
# set_axisを使って、列名を一括で新しく付け替えています
pop_projections_df = pop_projections_df.set_axis(
    [
        # リスト内包表記で、各列の1段目(i)と2段目(j)の名前を加工して結合
        (i + j.replace("Unnamed: 0_level_1", "和暦")  # 2段目が「Unnamed: 0...」なら「和暦」に置換
              .replace("Unnamed: 1_level_1", "西暦")) # 2段目が「Unnamed: 1...」なら「西暦」に置換
        .replace("　", "")                             # 全角スペースを削除してスッキリさせる
        for i, j in pop_projections_df.columns        # MultiIndex(2段組み)の列を一つずつ処理
    ],
    axis=1                                            # axis=1 は「列」に対して行う指定
)

In [62]:
pop_projections_df.head()

,年次和暦,年次西暦,"人口（1,000人）総数","人口（1,000人）0～14歳","人口（1,000人）15～64歳","人口（1,000人）65歳以上",割合（％）0～14歳,割合（％）15～64歳,割合（％）65歳以上
0,令和 2,2020,126146.099,15031.602,75087.865,36026.632,11.9,59.5,28.6
1,3,2021,125526.631,14792.434,74508.070,36226.126,11.8,59.4,28.9
2,4,2022,124977.644,14515.325,74196.042,36266.277,11.6,59.4,29.0
3,5,2023,124407.998,14202.024,73858.119,36347.856,11.4,59.4,29.2
4,6,2024,123843.669,13914.831,73465.788,36463.050,11.2,59.3,29.4


In [66]:
# 1. 縦に並べ替えたい（集約したい）数値列の名前をリストにする
# 「0～14歳」「15～64歳」「65歳以上」という3つの列が対象です
value_vars = ["人口（1,000人）0～14歳", "人口（1,000人）15～64歳", "人口（1,000人）65歳以上"]

# 2. pd.melt を使って、データを「縦持ち」に変換する
pop_tidy_df = pd.melt(
    pop_projections_df,        # もともとのデータフレーム（横に長い）
    id_vars="年次西暦",         # 固定する列（縦に並んだ後も、各行の「年」を識別するために残す）
    value_vars=value_vars,     # 縦にまとめたい列（上記で定義した3つの区分）
    var_name="年齢",            # 列名が並ぶ新しい列の名前を「年齢」にする
    value_name="人口"           # 数値（人口）が入る新しい列の名前を「人口」にする
)

# 3. 変換後のデータ末尾5行を確認する
pop_tidy_df.tail()

,年次西暦,年齢,人口
148,2066,"人口（1,000人）65歳以上",34861.121
149,2067,"人口（1,000人）65歳以上",34575.368
150,2068,"人口（1,000人）65歳以上",34272.879
151,2069,"人口（1,000人）65歳以上",33982.575
152,2070,"人口（1,000人）65歳以上",33671.450


In [68]:
# 1. 年ごとの総人口を算出する
# 年齢区分別に分かれていたデータを「年次西暦」でグループ化して合計し、
# 扱いやすいようにインデックスを振り直します
pop_totals = pop_tidy_df[["年次西暦", "人口"]].groupby("年次西暦").sum().reset_index()

# 2. 10年ごとのデータだけを抽出する
# 「年次西暦」を10で割った余りが0の年（2020, 2030, 2040年など）だけをピックアップします
totals_per_10year = pop_totals.loc[pop_totals["年次西暦"] % 10 == 0]

# 3. グラフに表示する用の文字ラベル（注釈）を作成する
# 人口（単位:千人）を1000で割って「百万人」単位にし、末尾に「百万」という文字を付けます
# 例: 125000（千人） → 125 → "125百万"
annotations = (totals_per_10year["人口"] // 1000).map(lambda x: f"{x:.0f}百万")

In [71]:
pop_projections_fig = px.bar(pop_tidy_df, x="年次西暦", y="人口", color="年齢")
pop_projections_fig.add_trace(go.Scatter(
    x=totals_per_10year["年次西暦"],
    y=totals_per_10year["人口"] + 5000,
    text=annotations,
    mode="text",
    showlegend=False
))
pop_projections_fig.update_layout(width=1000, height=400, font={"size": 18},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "千人"},
)
pop_projections_fig.update_layout(
    title_text="将来人口推計",
    title_x=0.5
)
pop_projections_fig.show()

## 世界の人口

- [World Population Prospects](https://population.un.org/wpp/)
- [World Population Prospects サイト 人口のグラフ](https://population.un.org/wpp/graphs)
- [download](https://population.un.org/wpp/downloads?folder=Standard%20Projections&group=CSV%20format)
- [国際標準化機構](https://www.iso.org/iso-3166-country-codes.html)


In [29]:
# data/ch07 フォルダにテキストp.127に書いてある方法でgzファイルをダウンロード＋展開してできたcsvファイルをコピーしてから実行する
import pandas as pd
from pathlib import Path

data_path = Path("data/ch07")
world_pop = pd.read_csv(data_path / "WPP2024_TotalPopulationBySex.csv", low_memory=False)
world_pop.columns

Index(['SortOrder', 'LocID', 'Notes', 'ISO3_code', 'ISO2_code', 'SDMX_code',
       'LocTypeID', 'LocTypeName', 'ParentID', 'Location', 'VarID', 'Variant',
       'Time', 'MidPeriod', 'PopMale', 'PopFemale', 'PopTotal', 'PopDensity'],
      dtype='object')

In [30]:
world_pop["ISO3_code"].unique()[-10:]

array(['PLW', 'ASM', 'COK', 'PYF', 'NIU', 'WSM', 'TKL', 'TON', 'TUV',
       'WLF'], dtype=object)

In [31]:
# 1. 抽出条件（フィルター）の作成
# 国コード (ISO3_code) が空でない行（特定の地域やグループではなく「国」のデータ）を選択
world_pop_cond = world_pop["ISO3_code"].notna()

# 予測のシナリオ（Variant）が「Medium（中位推計）」のものだけに絞り込む
world_pop_cond &= world_pop["Variant"] == "Medium"

# 2. 条件に合致するデータだけを抽出して新しい変数 pop_med に代入
pop_med = world_pop[world_pop_cond]

# 3. データの先頭5行について、「最初の5列」と「最後の5列」だけを指定して表示
# np.r_ を使うことで、離れた列（左端と右端）を同時につなげて指定しています
pop_med.iloc[:5, np.r_[0:5, len(pop_med.columns)-5:len(pop_med.columns)]]

,SortOrder,LocID,Notes,ISO3_code,ISO2_code,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
336230,64.0,108,NaN,BDI,BI,1950.5,1080.184,1174.755,2254.938,86.895
336231,64.0,108,NaN,BDI,BI,1951.5,1105.816,1199.930,2305.746,88.853
336232,64.0,108,NaN,BDI,BI,1952.5,1130.995,1224.809,2355.804,90.782
336233,64.0,108,NaN,BDI,BI,1953.5,1155.833,1249.353,2405.186,92.685
336234,64.0,108,NaN,BDI,BI,1954.5,1180.690,1273.896,2454.586,94.589


In [32]:
pop_med["Location"].nunique()

237

### 将来人口推移


In [33]:
# 1. 2020年のデータだけを抽出し、必要な列を絞り込む
# 「Time（年）」が2020年の行を選び、国コード・国名・総人口の3列だけを取り出します
pop2020 = pop_med.loc[pop_med["Time"] == 2020, ["ISO3_code", "Location", "PopTotal"]]

# 2. 人口が多い順にランキング（順位）を計算して新しい列を追加する
# rank(ascending=False) で、人口が多い方を1位（数値が小さい方）として順位付けします
pop2020 = pop2020.assign(**{"Rank2020": pop2020["PopTotal"].rank(ascending=False)})

# 3. 総人口(PopTotal)が多い順に上位20カ国を抜き出し、pop_rank に代入する
pop_rank = pop2020.nlargest(20, "PopTotal")

In [34]:
sorted_df = pop_rank.sort_values("PopTotal")
pop_bar_fig = go.Figure(go.Bar(
    x=sorted_df["PopTotal"], 
    y=sorted_df["Location"],
    text=sorted_df["PopTotal"].astype(int), 
    textposition="outside",
    orientation="h",
))
max_value = max(sorted_df["PopTotal"]) * 1.2  # 最大値の120%までx軸を拡張する
pop_bar_fig.update_layout(width=800, height=600, font={"size": 14},
    xaxis={
        "tickformat": ",.0f", 
        "ticksuffix": "千人",
        "title": "人口", 
        "range": [0, max_value]  # x軸の範囲を手動で設定
    },
    yaxis_title="国",
    margin={"l": 100, "r": 50}
)
pop_bar_fig.update_layout(
    title_text="世界の人口ランキング",
    title_x=0.5
)
pop_bar_fig.show()

In [35]:
# 1. データの準備：
# 全世界のデータ（pop_med）に、先ほど作った「上位20カ国リスト」を結合（merge）します。
# これにより、上位20カ国以外のデータが削ぎ落とされます。
pop_top20 = pop_med.merge(pop_rank[["Location", "Rank2020"]], on="Location", how="inner")

# 2. 並び替え：
# 2020年の順位（Rank2020）順に並べることで、グラフの凡例（右側のリスト）も順位通りに並びます。
pop_top20.sort_values(["Rank2020", "Time"], inplace=True)

# 3. カラーパレットの作成：
# 標準の10色セットでは20カ国分足りないため、2つのパレットを足して20色以上の色のリストを作ります。
pop_colors = plotly.colors.qualitative.Plotly + plotly.colors.qualitative.Set3

# 4. グラフの作成 (Plotly Expressを使用)：
pop_line_fig = px.line(
    pop_top20, 
    x="Time",           # 横軸：年
    y="PopTotal",       # 縦軸：人口
    color="Location",   # 色分け：国名ごと
    color_discrete_sequence=pop_colors # さきほど作った20色を使用
)

# 5. 見た目の調整：
pop_line_fig.update_layout(
    title_text="世界の人口推移",
    title_x=0.5,
    width=900, height=700, font={"size": 16},
    xaxis={"ticksuffix": "年", "title": None},        # 横軸の数字に「年」を付ける
    yaxis={"tickformat": ",.0f", "ticksuffix": "千人", "title": None} # 縦軸にカンマを入れ「千人」を付ける
)

# 6. 表示
pop_line_fig.show()

### 人口の世界地図

In [36]:
# px.choropleth という、地図（コロプレス図）を作る専用の関数を使っています
choropleth_fig = px.choropleth(
    pop2020,               # 2020年の人口データが入ったデータフレーム
    color="PopTotal",      # 「人口」の数値に応じて、色の濃さを変える
    locations="ISO3_code", # 各行が「どの国」かを示すコード（JPN, USAなど）を指定
    hover_name="Location"  # 地図上の国にマウスを乗せた時に表示される名前
)
choropleth_fig.update_layout(
    title_text="人口の世界地図",
    title_x=0.5,
    width=900
)
choropleth_fig.show()

In [37]:
pop_geo_fig = px.scatter_geo(
    pop_med, 
    size="PopTotal",  # 要素のサイズ
    locations="ISO3_code",  # 位置を ISO 3166-1 alpha-3 形式で指定
    hover_name="Location",  # ホバーツールのタイトル
    animation_frame="Time")  # アニメーション
pop_geo_fig.show()

## 生命表と寿命

- [生命表](https://www.mhlw.go.jp/toukei/list/list54-57.html)
- [公表](https://www.mhlw.go.jp/toukei/kouhyou/e-stat_55-18.xml)
- [標準生命表](https://www.actuaries.jp/lib/standard-life-table/)

### 完全生命表と簡易生命表


In [46]:
life_table_statsDataId = "0003109570" 
life_meta = get_metainfo(appId, life_table_statsDataId)
life_metadata = life_meta["GET_META_INFO"]["METADATA_INF"]
life_total_num = life_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
life_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


7792

In [47]:
life_data = get_statsdata(appId, life_table_statsDataId)
life_value = colname_to_japanese(cleansing_statsdata(life_data))
life_value.columns

e-Stat APIエラー (Status 0): 正常に終了しました。


Index(['表章項目コード', '年齢区分（完全生命表）コード', '性別コード', '時間軸（年次）コード', '単位', '値', '表章項目',
       '表章項目階層レベル', '表章項目単位', '年齢区分（完全生命表）', '年齢区分（完全生命表）階層レベル', '性別',
       '性別階層レベル', '時間軸（年次）', '時間軸（年次）階層レベル'],
      dtype='object')

In [48]:
life_value["時間軸（年次）"].unique()

array(['2020年', '2015年', '2010年', '2005年'], dtype=object)

In [49]:
life_cols = ["表章項目", "性別", "年齢区分（完全生命表）", "値"]
life_df = life_value.loc[life_value["時間軸（年次）"]=="2015年", life_cols]

In [50]:
life_df = life_df.assign(**{
    "年齢": life_df["年齢区分（完全生命表）"].str[:-1].astype(int),
    "週月年": life_df["年齢区分（完全生命表）"].str[-1]
})
life_df = life_df.loc[life_df["週月年"]=="年", ["表章項目", "性別", "年齢", "値"]]

In [51]:
# 1. データのピボット（縦持ちから横持ちへの変換）
# 「表章項目（平均余命、死亡率など）」を列の名前にし、
# 「性別」と「年齢」を行のラベル（見出し）にして、数値を配置します
pivot_df = life_df.pivot(columns="表章項目", index=["性別", "年齢"], values="値")

# 2. 列名のクリーニングと形式の整理（メソッドチェーンという書き方です）
pivot_df = (pivot_df
    # 列名に含まれる全角スペース（\u3000）をすべて削除して扱いやすくする
    .set_axis([c.replace("\u3000", "") for c in pivot_df.columns], axis=1)
    
    # 「性別」と「年齢」をインデックス（見出し）から普通の列に戻す
    # これにより、pivot_df["性別"] のようにアクセスできるようになります
    .reset_index()
)

# 3. 先頭5行を表示して、正しく変換されたか確認する
pivot_df.head()

,性別,年齢,定常人口(Tx),定常人口(nLx),平均余命(ex),死力(μx),死亡数(ndx),死亡率(nqx),生存数(lx),生存率(npx)
0,女,0,8698726.0,99861.0,86.99,0.05782,178.0,0.00178,100000.0,0.99822
1,女,1,8598865.0,99806.0,86.14,0.00040,32.0,0.00032,99822.0,0.99968
2,女,2,8499059.0,99780.0,85.17,0.00023,20.0,0.00020,99790.0,0.99980
3,女,3,8399279.0,99763.0,84.19,0.00016,12.0,0.00012,99770.0,0.99988
4,女,4,8299516.0,99753.0,83.20,0.00010,8.0,0.00008,99758.0,0.99992


In [114]:
# 1. 0歳児の平均余命（＝平均寿命）を特定する関数を定義
def calc_leb(df, gender):
    # df.loc[...] を使って、特定の行と列をピンポイントで指定
    # 条件1: 年齢が 0 歳であること
    # 条件2: 性別が指定されたもの（男または女）であること
    # 取得する列: 「平均余命(ex)」
    return df.loc[
        (df["年齢"] == 0) & (df["性別"] == gender), "平均余命(ex)"
    ].iat[0] # .iat[0] を使うことで、データフレームという「箱」から数値のみを取り出す

# 2. 関数を使って、男性の平均寿命を取得
lifespan_male = calc_leb(pivot_df, "男")

# 3. 関数を使って、女性の平均寿命を取得
lifespan_female = calc_leb(pivot_df, "女")

# 4. 結果を表示
print("男:", lifespan_male, ", 女:", lifespan_female)

男: 80.75 , 女: 86.99


In [117]:
# 1. 寿命中位数（半数の人が生存している年齢）を計算する関数を定義
def calc_med_lifespan(df, gender):
    # 指定された性別のデータを取り出し、年齢順に並べます
    data = df[df["性別"] == gender].sort_values("年齢")
    
    # 0歳の時の生存数（通常は100,000人）のちょうど半分（50%）の値を計算します
    # これが「生存者が半分になるライン」です
    half = data["生存数(lx)"].iloc[0] / 2
    
    # 厳密な計算（比例配分など）はせず、
    # 生存数が半分（half）を下回った瞬間の「年齢」を最初の1件だけ取得して返します
    return data[data["生存数(lx)"] < half]["年齢"].iloc[0]

# 2. 男性と女性それぞれの寿命中位数を計算して表示
print("男:", calc_med_lifespan(pivot_df, "男"), ", 女:", calc_med_lifespan(pivot_df, "女"))

男: 84 , 女: 90


In [120]:
ndx = pivot_df[["性別", "年齢", "死亡数(ndx)"]]
male_ndx = ndx.loc[ndx["性別"]=="男"]
female_ndx = ndx.loc[ndx["性別"]=="女"]
life_fig = go.Figure(data=[
    go.Bar(x=male_ndx["年齢"], y=male_ndx["死亡数(ndx)"], name="男性"),
    go.Bar(x=female_ndx["年齢"], y=female_ndx["死亡数(ndx)"], name="女性")
])
life_fig.add_trace(go.Scatter(
    x=[lifespan_male, lifespan_male], 
    y=[0, 5000],
    name="男性平均寿命", 
    mode="lines", line={"dash": "dash"},
    marker_color=px.colors.qualitative.Plotly[0]))
life_fig.add_trace(go.Scatter(
    x=[lifespan_female, lifespan_female], 
    y=[0, 5000],
    name="女性平均寿命", 
    mode="lines", line={"dash": "dash"},
    marker_color=px.colors.qualitative.Plotly[1]))
life_fig.update_layout(width=800, height=450, font={"size": 18},
    xaxis={"ticksuffix": "歳", "title": None},
    yaxis={"tickformat": ",.0f", "ticksuffix": "人", "title": None},
    title="10万人あたり死亡数", 
    barmode="group"
)
life_fig.update_layout(
    title_x=0.5
)
life_fig.show()

### p.135 世界の平均寿命・健康寿命

- [GHO](https://www.who.int/data/gho)
- [OData API](https://www.who.int/data/gho/info/gho-odata-api)
- [Terms and conditions for WHO datasets](https://data.who.int/about/data/terms-and-conditions)


In [ ]:
# 1. WHO GHO APIのベースURL（データの取得元アドレス）を設定
base_url = "https://ghoapi.azureedge.net/api/"

# 2. 検索したいキーワード（ここでは「平均寿命」）を指定
contain_name = "Life expectancy"

# 3. OData（APIの規格）形式でフィルタ条件を作成
# 「IndicatorName（指標名）に contain_name の文字列が含まれるものを抽出する」という命令
contain_params = {"$filter": f"contains(IndicatorName,'{contain_name}')"}

# 4. 「Indicator（指標一覧）」エンドポイントに対して、フィルタ条件付きでリクエストを送信
# requests.get(url, params=...) という関数の params 引数が、「URLの末尾につけるパラメータ（?key=value の部分）」を辞書形式で受け取るルールになっている。
indicator_contain_res = requests.get(base_url + "Indicator", params=contain_params)

# 5. 取得したレスポンスをJSON形式のデータとして出力（中身を確認する）
indicator_contain_res.json()

# このコードのポイント
# $filter: APIから送られてくる大量のデータの中から、必要なものだけを絞り込むためのパラメータです。
# contains(フィールド名, '文字列'): 指定したフィールドに特定の文字列が含まれているかどうかを判定する関数です。
# 実行結果: Life expectancy at birth（出生時平均寿命）や Healthy life expectancy（健康寿命）といった、キーワードに合致する指標のIDや名前がリストとして返ってきます。

{'@odata.context': 'https://ghoapi.azureedge.net/api/$metadata#Indicator',
 'value': [{'IndicatorCode': 'WHOSIS_000002',
   'IndicatorName': 'Healthy life expectancy (HALE) at birth (years)',
   'Language': 'EN'},
  {'IndicatorCode': 'WHOSIS_000007',
   'IndicatorName': 'Healthy life expectancy (HALE) at age 60 (years)',
   'Language': 'EN'},
  {'IndicatorCode': 'WHOSIS_000015',
   'IndicatorName': 'Life expectancy at age 60 (years)',
   'Language': 'EN'},
  {'IndicatorCode': 'WHOSIS_000001',
   'IndicatorName': 'Life expectancy at birth (years)',
   'Language': 'EN'}]}

In [10]:
# 1. 取得したい指標の「正確な名前」を指定
# （前回は部分一致でしたが、今回は完全一致で指定しています）
eq_name = "Life expectancy at birth (years)"

# 2. フィルタ条件として「IndicatorName が eq_name と等しい(eq)」を設定
# これにより、特定の指標1つだけに絞り込めます
eq_params = {"$filter": f"IndicatorName eq '{eq_name}'"}

# 3. 指定した名前の指標情報を取得するためにリクエストを送信
indicator_eq_res = requests.get(base_url + "Indicator", params=eq_params)

# 4. 取得結果（JSON形式）を表示して、指標のID（IndicatorCode）などを確認する
indicator_eq_res.json()

{'@odata.context': 'https://ghoapi.azureedge.net/api/$metadata#Indicator',
 'value': [{'IndicatorCode': 'WHOSIS_000001',
   'IndicatorName': 'Life expectancy at birth (years)',
   'Language': 'EN'}]}

In [11]:
lifeexp_res = requests.get(base_url + "WHOSIS_000001")
lifeexp_value = pd.json_normalize(lifeexp_res.json(), record_path=["value"])
lifeexp_value.columns

Index(['Id', 'IndicatorCode', 'SpatialDimType', 'SpatialDim',
       'ParentLocationCode', 'TimeDimType', 'ParentLocation', 'Dim1Type',
       'TimeDim', 'Dim1', 'Dim2Type', 'Dim2', 'Dim3Type', 'Dim3',
       'DataSourceDimType', 'DataSourceDim', 'Value', 'NumericValue', 'Low',
       'High', 'Comments', 'Date', 'TimeDimensionValue', 'TimeDimensionBegin',
       'TimeDimensionEnd'],
      dtype='object')

In [12]:
lifeexp_value["TimeDim"].unique()

array([2012, 2002, 2011, 2001, 2008, 2014, 2016, 2006, 2020, 2021, 2010,
       2007, 2015, 2009, 2018, 2013, 2017, 2000, 2004, 2005, 2019, 2003])

In [13]:
lifeexp_value["Dim1"].unique()

array(['SEX_BTSX', 'SEX_MLE', 'SEX_FMLE'], dtype=object)

In [ ]:
lifeexp_value["SpatialDim"].unique()    # 国名ISOコードを表示する

array(['HUN', 'HTI', 'VUT', 'GNB', 'SSD', 'DNK', 'IRQ', 'EUR', 'TTO',
       'OMN', 'SLV', 'BRA', 'SYC', 'SEN', 'DZA', 'SVK', 'ESP', 'COM',
       'MLI', 'MUS', 'ZMB', 'URY', 'CAN', 'USA', 'AMR', 'JOR', 'FJI',
       'QAT', 'IDN', 'JPN', 'ETH', 'AFR', 'SYR', 'MWI', 'FSM', 'BHR',
       'BLR', 'BRN', 'AUT', 'EGY', 'SGP', 'MYS', 'BFA', 'NER', 'NZL',
       'SDN', 'LAO', 'COG', 'HRV', 'ECU', 'PRY', 'IRN', 'AUS', 'SVN',
       'VEN', 'BLZ', 'HND', 'CHE', 'PAN', 'MDG', 'GRC', 'BIH', 'GBR',
       'JAM', 'BGR', 'PNG', 'ATG', 'WB_UMI', 'BHS', 'WB_HI', 'SRB',
       'GLOBAL', 'AZE', 'GTM', 'SAU', 'CYP', 'GHA', 'ZAF', 'BEL', 'RWA',
       'PSE', 'SEAR', 'AGO', 'MLT', 'BRB', 'DJI', 'CZE', 'LUX', 'CRI',
       'NIC', 'GEO', 'UKR', 'TUR', 'LBR', 'ERI', 'SUR', 'GNQ', 'PER',
       'LTU', 'CMR', 'TJK', 'ISR', 'MAR', 'GIN', 'NOR', 'TKM', 'TLS',
       'SWZ', 'KWT', 'TCD', 'AFG', 'UZB', 'YEM', 'WB_LMI', 'MNG', 'DOM',
       'EMR', 'LKA', 'FIN', 'CIV', 'COD', 'LBN', 'MEX', 'IND', 'WB_LI',
       'NGA',

In [ ]:
# 1. 絞り込み条件1：調査年が「2020年」のデータだけを指定
lifeexp_cond = lifeexp_value["TimeDim"] == 2020

# 2. 絞り込み条件2を追加：性別が「男女計（SEX_BTSX）」のデータだけを指定
# &= は「かつ（AND）」という意味。これにより「2020年 かつ 男女計」に絞られます
lifeexp_cond &= lifeexp_value["Dim1"] == "SEX_BTSX"

# 3. 作成した条件(lifeexp_cond)を使い、特定の列だけを抽出して新しいDataFrameを作成
# 対象列：Dim1(性別), SpatialDim(国コード), NumericValue(寿命の数値)
lifeexp_df = lifeexp_value.loc[lifeexp_cond, ["Dim1", "SpatialDim", "NumericValue"]]

# 4. 整理されたデータの先頭5行を表示して確認
lifeexp_df.head()

,Dim1,SpatialDim,NumericValue
11,SEX_BTSX,OMN,73.886681
43,SEX_BTSX,MUS,73.569754
147,SEX_BTSX,TKM,68.473669
290,SEX_BTSX,SDN,68.440770
325,SEX_BTSX,ECU,72.711382


##### 覚えておくと便利なポイント
- `&=` : 複数の条件を重ねる（絞り込む）ときに使います。これを使うことで、条件式を1行ずつ綺麗に書き足していくことができます。
- `Dim1` の中身 :
    - `SEX_BTSX`: Both Sexes（男女計）
    - `SEX_MLE`: Male（男性）
    - `SEX_FMLE`: Female（女性） が一般的です。今回は分析をシンプルにするために「男女計」を選んでいます。
- `loc` の役割 : `loc[行の条件, 列のリスト]` という書き方で、「欲しいデータ」を正確に切り出しています。

In [ ]:
# 1. 人口データ(pop2020)と平均寿命データ(lifeexp_df)を「国コード」をキーにして結合(マージ)
# left_on="ISO3_code": 人口データ側の国コード列を指定
# right_on="SpatialDim": 寿命データ側の国コード列を指定
# how="inner": 両方のデータに存在する国のみを残す(内部結合)
country_df = pop2020.merge(lifeexp_df, left_on="ISO3_code", right_on="SpatialDim", how="inner")

# 2. 人口("PopTotal")が多い順に上位30件を抽出し、必要な列だけに整理
# nlargest(30, "PopTotal"): 人口が多い順に30行取得
# loc[:, [... ]]: 「国名」「総人口」「平均寿命」の3列のみを選択
top30_df = country_df.nlargest(30, "PopTotal").loc[:, ["Location", "PopTotal", "NumericValue"]]
top30_df

,Location,PopTotal,NumericValue
59,China,1426106.093,77.481135
67,India,1402617.695,70.186909
174,United States of America,339436.159,76.892509
75,Indonesia,274814.866,69.703673
71,Pakistan,235001.746,65.849484
50,Nigeria,213996.181,62.835404
163,Brazil,208660.842,74.113936
65,Bangladesh,166298.024,73.298130
109,Russian Federation,146371.299,71.377480
158,Mexico,126799.054,70.946252


In [57]:
lifeexp_fig = px.scatter(
    x=top30_df["PopTotal"], 
    y=top30_df["NumericValue"], 
    text=top30_df["Location"]
)
lifeexp_fig.update_traces(textposition="bottom right")
lifeexp_fig.update_layout(width=800, height=600, font={"size": 14},
    xaxis={
        "tickformat": ",.0f", 
        "ticksuffix": "千人", 
        "title": "人口", 
        "range": [0, top30_df["PopTotal"].max() * 1.1]
    },
    yaxis={"ticksuffix": "歳", "title": "平均寿命"}
)
lifeexp_fig.update_layout(
    title="平均寿命と人口の関係",
    title_x=0.5
)
lifeexp_fig.show()

In [58]:
contain_healthy = "Healthy"
healthy_params = {"$filter": f"contains(IndicatorName,'{contain_name}')"}
healthy_res = requests.get(base_url + "Indicator", params=contain_params)
healthy_res.json()

{'@odata.context': 'https://ghoapi.azureedge.net/api/$metadata#Indicator',
 'value': [{'IndicatorCode': 'WHOSIS_000002',
   'IndicatorName': 'Healthy life expectancy (HALE) at birth (years)',
   'Language': 'EN'},
  {'IndicatorCode': 'WHOSIS_000007',
   'IndicatorName': 'Healthy life expectancy (HALE) at age 60 (years)',
   'Language': 'EN'},
  {'IndicatorCode': 'WHOSIS_000015',
   'IndicatorName': 'Life expectancy at age 60 (years)',
   'Language': 'EN'},
  {'IndicatorCode': 'WHOSIS_000001',
   'IndicatorName': 'Life expectancy at birth (years)',
   'Language': 'EN'}]}

In [59]:
hale_res = requests.get(base_url + "WHOSIS_000002")
hale_value = pd.json_normalize(hale_res.json(), record_path=["value"])
cond = hale_value["TimeDim"] == 2020
cond &= hale_value["Dim1"] != "BTSX"
hale_df = hale_value.loc[cond, ["Dim1", "SpatialDim", "NumericValue"]]
hale_df.head()

,Dim1,SpatialDim,NumericValue
9,SEX_FMLE,JPN,75.113417
14,SEX_FMLE,IRN,64.698292
28,SEX_MLE,VEN,60.244560
33,SEX_BTSX,YEM,57.039044
47,SEX_FMLE,SWE,70.851427



- [保健指標評価研究所]((https://www.healthdata.org/))
- [ihme_data](https://ghdx.healthdata.org/ihme_data)
- [terms-and-conditions](https://www.healthdata.org/data-tools-practices/data-practices/terms-and-conditions)


## p.139 疾病と感染症

### 人口動態と死因の国際疾病分類

- [International
Statistical Classification of Diseases and Related Health Problems：ICD](https://www.mhlw.go.jp/toukei/sippei/)

#### 死因年次推移


In [60]:
disease_statsDataId = "0003411656"
disease_meta = get_metainfo(appId, disease_statsDataId)
disease_metadata = disease_meta["GET_META_INFO"]["METADATA_INF"]
disease_total_num = disease_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
disease_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


12444

In [61]:
disease_data = get_statsdata(appId, disease_statsDataId)
disease_value = colname_to_japanese(cleansing_statsdata(disease_data))
disease_value.columns

e-Stat APIエラー (Status 0): 正常に終了しました。


Index(['表章項目コード', '死因年次推移分類コード', '性別コード', '時間軸(年次)コード', '単位', '値', '表章項目',
       '表章項目階層レベル', '表章項目単位', '死因年次推移分類', '死因年次推移分類階層レベル', '死因年次推移分類親コード',
       '性別', '性別階層レベル', '性別親コード', '時間軸(年次)', '時間軸(年次)階層レベル'],
      dtype='object')

In [62]:
disease_value["死因年次推移分類"].unique()

array(['総数', 'Hi01_結核', 'Hi02_悪性新生物＜腫瘍＞', 'Hi03_糖尿病', 'Hi04_高血圧性疾患',
       'Hi05_心疾患（高血圧性を除く）', 'Hi06_脳血管疾患', 'Hi07_肺炎', 'Hi08_慢性気管支炎及び肺気腫',
       'Hi09_喘息', 'Hi10_胃潰瘍及び十二指腸潰瘍', 'Hi11_肝疾患', 'Hi12_腎不全', 'Hi13_老衰',
       'Hi14_不慮の事故', 'Hi15_（再掲）交通事故', 'Hi16_自殺'], dtype=object)

In [63]:
disease_value["単位"].unique()

array(['人', '人口10万対'], dtype=object)

In [64]:
disease_value["性別"].unique()

array(['総数', '男', '女'], dtype=object)

In [67]:
# 1. データの絞り込み条件を作成
# 条件1：死因の「総数」は除外（個別の死因のみに注目するため）
disease_cond = disease_value["死因年次推移分類"] != "総数" 
# 条件2：単位を「人口10万対（死亡率）」に指定
disease_cond &= disease_value["単位"] == "人口10万対"
# 条件3：性別を「総数」に指定
disease_cond &= disease_value["性別"] == "総数"

# 2. 条件を適用し、必要な列だけ抽出して並び替える
disease_df = disease_value.loc[
    disease_cond, ["死因年次推移分類", "時間軸(年次)コード", "値"]
].sort_values(["時間軸(年次)コード", "死因年次推移分類"])

# 3. 「時間軸コード」から「年（西暦）」を計算して新しい列を作成
# 1899000000 などの長いコードの先頭4桁が年を表しているため
# 100万で割る（//）ことで西暦部分のみを整数として取り出している
disease_df = disease_df.assign(**{
    "年": disease_df["時間軸(年次)コード"].astype(int) // 1_000_000     # 例: 1899000000 -> 1899
})

# 4. 出来上がったデータの先頭部分を表示
disease_df.head()

,死因年次推移分類,時間軸(年次)コード,値,年
6709,Hi01_結核,1899000000,155.7,1899
7075,Hi02_悪性新生物＜腫瘍＞,1899000000,44.7,1899
7441,Hi03_糖尿病,1899000000,NaN,1899
7807,Hi04_高血圧性疾患,1899000000,NaN,1899
8173,Hi05_心疾患（高血圧性を除く）,1899000000,48.6,1899


In [68]:
disease_colors = px.colors.qualitative.Plotly + px.colors.qualitative.Set3
disease_fig = px.line(disease_df, x="年", y="値",
    color="死因年次推移分類", color_discrete_sequence=disease_colors)
disease_fig.update_layout(width=900, height=600, font={"size": 16},
    xaxis={"ticksuffix": "年", "title": None},
    yaxis={"ticksuffix": "人", "tickformat": ",.0f", "title": "人口10万対"}
)
disease_fig.show()

#### 年齢ごと死因


In [69]:
icd_statsDataId = "0003411691"
icd_meta = get_metainfo(appId, icd_statsDataId)
icd_metadata = icd_meta["GET_META_INFO"]["METADATA_INF"]
icd_total_num = icd_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
icd_total_num

e-Stat APIエラー (Status 0): 正常に終了しました。


348432

In [75]:
# 1. どんな名前のキーが入っているか確認
print(icd_metadata.keys())

dict_keys(['TABLE_INF', 'CLASS_INF'])


In [76]:
# 1. CLASS_INF の中にある CLASS_OBJ を取得
class_obj = icd_metadata["CLASS_INF"]["CLASS_OBJ"]

# 2. カテゴリーの一覧を確認
for obj in class_obj:
    print(f"ID: {obj['@id']}, 名前: {obj['@name']}")

ID: tab, 名前: 表章項目
ID: cat01, 名前: 年齢(5歳階級)
ID: cat02, 名前: 性別
ID: cat03, 名前: 死因基本分類_1
ID: time, 名前: 時間軸(年次)


In [78]:
# class_obj[3] が「死因基本分類_1」のはずです
death_causes = class_obj[3]["CLASS"]

# レベル2の項目を表示
print("--- レベル2の例 ---")
count = 0
for item in death_causes:
    if item["@level"] == "2" and count < 5:
        print(item["@name"])
        count += 1

# レベル3の項目を表示
print("\n--- レベル3の例 ---")
count = 0
for item in death_causes:
    if item["@level"] == "3" and count < 5:
        print(item["@name"])
        count += 1

--- レベル2の例 ---
第1章　感染症及び寄生虫症（A00－B99）
第2章　新生物＜腫瘍＞（C00-D48）
第3章　血液及び造血器の疾患並びに免疫機構の障害（D50－D89）
第4章　内分泌，栄養及び代謝疾患（E00-E90）

--- レベル3の例 ---
腸管感染症(A00-A09)
結核(A15-A19)
人畜共通細菌性疾患(A20-A28)
その他の細菌性疾患(A30-A49)
主として性的伝播様式をとる感染症(A50-A64)


In [ ]:
%%time
# 1. 取得したい統計表IDのリスト（複数の表からデータを集める）
icd_statsDataIds = ["0003411691", "0003411692", "0003411693", "0004018620"]

# 2. APIへの共通パラメータ設定
# lvCat03: "3"（死因分類をレベル3の詳細さで取得）
# cdTime: "2023000000"（2023年のデータを指定）
icd_params = {"lvCat03": "3", "cdTime": "2023000000"}

cat = "死因基本分類"
icd_dfs = [] # 各テーブルのデータを格納するリスト

# 3. 指定したIDのリストを順番に処理
for n, sdi in enumerate(icd_statsDataIds):
    print(sdi, "取得開始")
    
    # APIからデータを取得
    icd_data = get_statsdata(appId, sdi, params=icd_params)
    # データをクレンジングし、列名を日本語に変換
    icd_value = colname_to_japanese(cleansing_statsdata(icd_data))
    
    # 抽出する列を指定（テーブルごとに異なる末尾の番号に対応させる）
    icd_cols = ["年齢(5歳階級)", "性別", f"死因基本分類_{n+1}", "値"]
    
    # 4. 絞り込み条件の作成
    # 「性別：総数」を除外
    icd_cond = icd_value["性別"] != "総数"
    # 「年齢：総数」および「不詳」を除外
    icd_cond &= ~icd_value["年齢(5歳階級)"].isin(["総数", "不詳"])
    
    # 条件を適用し、列名を共通の「死因基本分類」にリネームして保存
    icd_df = icd_value.loc[icd_cond, icd_cols].rename(columns={f"{cat}_{n+1}": cat})
    icd_dfs += [icd_df]

# 5. 全てのテーブルから取得したデータを縦に連結（合体）
icd = pd.concat(icd_dfs)
icd.head()

0003411691 取得開始
e-Stat APIエラー (Status 0): 正常に終了しました。
0003411692 取得開始
e-Stat APIエラー (Status 0): 正常に終了しました。
0003411693 取得開始
e-Stat APIエラー (Status 0): 正常に終了しました。
0004018620 取得開始
e-Stat APIエラー (Status 0): 正常に終了しました。
CPU times: total: 1.44 s
Wall time: 6.48 s


,年齢(5歳階級),性別,死因基本分類,値
152,0歳,男,腸管感染症(A00-A09),1.0
153,0歳,男,結核(A15-A19),NaN
154,0歳,男,人畜共通細菌性疾患(A20-A28),NaN
155,0歳,男,その他の細菌性疾患(A30-A49),10.0
156,0歳,男,主として性的伝播様式をとる感染症(A50-A64),2.0


##### このコードのポイント
1. `f"死因基本分類_{n+1}"`: e-StatのAPIは、テーブルが異なると「死因基本分類_1」「死因基本分類_2」のように列名が微妙に変わることがあります。それを`enumerate` のインデックス`n` を使って自動的に対応させています。
2. `~...isin([...])`: 「総数」と「不詳」の両方を除外するために、`isin`（リストに含まれるか）と`~`（否定）を組み合わせてスマートに書いています。
3. 効率的なデータ収集: 本来なら一つずつ手作業でダウンロードする複数の統計表を、スクリプトで一気に取得・結合することで、分析の手間を大幅に削減しています。

これで、性別ごと・年齢階級ごとの詳細な死因分析ができる「整ったデータ」が完成します！

In [81]:
age_dict = {
    "0～4歳": ["0～4歳"],
    "5～9歳": ["5～9歳"],
    "10～19歳": ["10～14歳", "15～19歳"],
    "20～29歳": ["20～24歳", "25～29歳"],
    "30～39歳": ["30～34歳", "35～39歳"],
    "40～49歳": ["40～44歳", "45～49歳"],
    "50～69歳": ["50～54歳", "55～59歳", "60～64歳", "65～69歳"],
    "70～89歳": ["70～74歳", "75～79歳", "80～84歳", "85～89歳"],
    "90～歳": ["90～94歳", "95～99歳", "100歳以上"],
}

In [82]:
def top_causes_by_age(icd, age_dict, cat, sex, top_n=5):
    death_cause = []
    for age_label, age_list in age_dict.items():
        cond = icd["年齢(5歳階級)"].isin(age_list) & (icd["性別"] == sex)
        icd_rank = (icd
            .loc[cond, [cat, "値"]]
            .groupby(cat, as_index=False).sum()
            .sort_values("値", ascending=False)
        )
        top = icd_rank[cat].head(top_n).reset_index(drop=True)
        top.name = age_label
        death_cause.append(top)
    return pd.concat(death_cause, axis=1)
top_causes_by_age(icd, age_dict, cat, sex="男")

,0～4歳,5～9歳,10～19歳,20～29歳,30～39歳,40～49歳,50～69歳,70～89歳,90～歳
0,診断名不明確及び原因不明の死亡(R95-R99),悪性新生物＜腫瘍＞(C00-C96),故意の自傷及び自殺(X60-X84),故意の自傷及び自殺(X60-X84),故意の自傷及び自殺(X60-X84),故意の自傷及び自殺(X60-X84),悪性新生物＜腫瘍＞(C00-C96),悪性新生物＜腫瘍＞(C00-C96),全身症状及び徴候(R50-R68)
1,循環器系の先天奇形(Q20-Q28),不慮の事故(V01-X59),外因のその他及び詳細不明の作用(T66-T78),外因のその他及び詳細不明の作用(T66-T78),外因のその他及び詳細不明の作用(T66-T78),悪性新生物＜腫瘍＞(C00-C96),虚血性心疾患(I20-I25),その他の型の心疾患(I30-I51),悪性新生物＜腫瘍＞(C00-C96)
2,周産期に特異的な呼吸障害及び心血管障害(P20-P29),外因のその他及び詳細不明の作用(T66-T78),不慮の事故(V01-X59),不慮の事故(V01-X59),悪性新生物＜腫瘍＞(C00-C96),外因のその他及び詳細不明の作用(T66-T78),脳血管疾患(I60-I69),脳血管疾患(I60-I69),その他の型の心疾患(I30-I51)
3,染色体異常，他に分類されないもの(Q90-Q99),インフルエンザ及び肺炎(J09-J18),悪性新生物＜腫瘍＞(C00-C96),薬用を主としない物質の毒作用(T51-T65),薬用を主としない物質の毒作用(T51-T65),脳血管疾患(I60-I69),その他の型の心疾患(I30-I51),インフルエンザ及び肺炎(J09-J18),インフルエンザ及び肺炎(J09-J18)
4,不慮の事故(V01-X59),その他の型の心疾患(I30-I51),頭部損傷(S00-S09),悪性新生物＜腫瘍＞(C00-C96),不慮の事故(V01-X59),虚血性心疾患(I20-I25),診断名不明確及び原因不明の死亡(R95-R99),虚血性心疾患(I20-I25),外的因子による肺疾患(J60-J70)


In [83]:
top_causes_by_age(icd, age_dict, cat, sex="女")

,0～4歳,5～9歳,10～19歳,20～29歳,30～39歳,40～49歳,50～69歳,70～89歳,90～歳
0,染色体異常，他に分類されないもの(Q90-Q99),悪性新生物＜腫瘍＞(C00-C96),故意の自傷及び自殺(X60-X84),故意の自傷及び自殺(X60-X84),悪性新生物＜腫瘍＞(C00-C96),悪性新生物＜腫瘍＞(C00-C96),悪性新生物＜腫瘍＞(C00-C96),悪性新生物＜腫瘍＞(C00-C96),全身症状及び徴候(R50-R68)
1,循環器系の先天奇形(Q20-Q28),不慮の事故(V01-X59),外因のその他及び詳細不明の作用(T66-T78),外因のその他及び詳細不明の作用(T66-T78),故意の自傷及び自殺(X60-X84),故意の自傷及び自殺(X60-X84),脳血管疾患(I60-I69),全身症状及び徴候(R50-R68),その他の型の心疾患(I30-I51)
2,周産期に特異的な呼吸障害及び心血管障害(P20-P29),インフルエンザ及び肺炎(J09-J18),悪性新生物＜腫瘍＞(C00-C96),悪性新生物＜腫瘍＞(C00-C96),外因のその他及び詳細不明の作用(T66-T78),外因のその他及び詳細不明の作用(T66-T78),その他の型の心疾患(I30-I51),その他の型の心疾患(I30-I51),悪性新生物＜腫瘍＞(C00-C96)
3,診断名不明確及び原因不明の死亡(R95-R99),染色体異常，他に分類されないもの(Q90-Q99),頭部損傷(S00-S09),薬用を主としない物質の毒作用(T51-T65),不慮の事故(V01-X59),脳血管疾患(I60-I69),故意の自傷及び自殺(X60-X84),脳血管疾患(I60-I69),脳血管疾患(I60-I69)
4,不慮の事故(V01-X59),頭部損傷(S00-S09),不慮の事故(V01-X59),頭部損傷(S00-S09),その他の型の心疾患(I30-I51),肝疾患(K70-K76),外因のその他及び詳細不明の作用(T66-T78),虚血性心疾患(I20-I25),インフルエンザ及び肺炎(J09-J18)


### p.144 COVID-19


#### 厚生労働省のCOVID-19データ

- [厚生労働省のCOVID-19 のオープンデータのサイト](https://www.mhlw.go.jp/stf/covid-19/open-data.html)


In [84]:
covid_path = data_path / "severe_cases_daily.csv"
#covid_path = "https://covid19.mhlw.go.jp/public/opendata/severe_cases_daily.csv"
covid_df = pd.read_csv(covid_path, parse_dates=["Date"])
covid_tidy = (covid_df[["Date", "ALL"]]
    .rename(columns={"ALL": "重症者数の推移"})
    .set_index("Date")
    .resample("MS").sum()
    .reset_index()
)
covid_tidy.head()

,Date,重症者数の推移
0,2020-05-01,4120
1,2020-06-01,2052
2,2020-07-01,1474
3,2020-08-01,5895
4,2020-09-01,5269


#### WHOのCOVID-19データ

- [WHO のサイト](https://data.who.int/dashboards/covid19/data)


In [ ]:
csvname = "WHO-COVID-19-global-daily-data.csv"
who_covid_data = pd.read_csv(data_path / csvname, parse_dates=["Date_reported"])    # parse_dates=["Date_reported"]でDate_reportedをdatetime型に変換
who_covid_data.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-04,AF,Afghanistan,EMR,NaN,0,NaN,0
1,2020-01-04,DZ,Algeria,AFR,NaN,0,NaN,0
2,2020-01-04,AL,Albania,EUR,NaN,0,NaN,0
3,2020-01-04,AI,Anguilla,AMR,NaN,0,NaN,0
4,2020-01-04,AS,American Samoa,WPR,0.0,0,0.0,0


In [89]:
# 1. 各国の「最終的な累計死亡数」を取得する
max_deaths = (who_covid_data
    # 日付順に並び替えてから、国名と累計死亡数の列だけにする
    .sort_values("Date_reported")[["Country", "Cumulative_deaths"]]
    # 国ごとにグループ化し、一番最後の行（＝最新データ）を取り出す
    .groupby("Country").last()
    .reset_index()
)

# 2. 累計死亡数が多い「上位25カ国」を抽出する
rank_deaths = max_deaths.nlargest(25, "Cumulative_deaths")

# 3. 上位25カ国のリストを使って、元の「日別データ」を絞り込む
# rank_deaths（上位25国リスト）と元のデータをマージ(結合)することで実現
who_covid_daily = rank_deaths.merge(who_covid_data, on="Country", suffixes=("_rank", ""))

# 4. 「日別」のデータを「月別」の合計値に集計し直す
who_covid_monthly = (who_covid_daily[["Date_reported", "Country", "New_deaths"]]
    # Date_reportedを「月末(ME)」単位でまとめ、さらに国ごとにグループ化
    .groupby([pd.Grouper(key="Date_reported", freq="ME"), "Country"]).sum()
    .reset_index()
)

In [90]:
# 1. Plotly Expressで折れ線グラフ(line)を作成
covid_fig = px.line(
    who_covid_monthly,          # 使用するデータ（月別集計データ）
    x="Date_reported",          # 横軸：報告日（月単位）
    y="New_deaths",             # 縦軸：新規死亡者数
    color="Country",            # 色分け：国ごと（国ごとに別の線になる）
    # カラーパレット：「Alphabet」という26色の豊富な色セットを使用
    color_discrete_sequence=px.colors.qualitative.Alphabet
)

# 2. グラフのレイアウト（見た目）を細かく更新
covid_fig.update_layout(
    width=1200, height=800,     # グラフのサイズ（幅1200、高さ800ピクセル）
    font={"size": 16},          # 全体の文字サイズを16に設定
    xaxis={"ticksuffix": "年"}, # 横軸のラベルの末尾に「年」を表示（例：2021年）
    yaxis={
        "tickformat": ",.0f",   # 縦軸の数字を 3桁区切りのカンマ付きにする（例：1,000）
        "ticksuffix": "人"      # 縦軸の末尾に「人」を表示（例：100人）
    }
)

# 3. グラフを表示する（マウスを乗せると値が見える動的なグラフになる）
covid_fig.show()